In [216]:
import os
from os.path import commonprefix
from functools import reduce
from collections import defaultdict
import re

import numpy as np
import pandas as pd
from scipy import stats
from Bio import SeqIO

import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [217]:
# Set figure parameters
sns.set_style('ticks')
sns.set_context('paper')
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['xtick.labelsize'] = 6
mpl.rcParams['ytick.labelsize'] = 6

colours = {'DNA': '#fab95b', 
           'LTR': '#665c84', 
           'LINE': '#71a0a5', 
           'RC': '#ED7F35',
           'SINE': '#5E96B5',
           'Other': '#212121'}

# Aggregation and summary of results
Since many salient features of the _Danio rerio_ transposable element landscape are calculated from different sources, in this notebook I have aggregated these and produced some summary figures.

Age estimates are calculated by two methods, the first being percent sequence divergence from consensus, and the second being terminal branch lengths from phylogenetic trees. The former is taken from parsed RepeatMasker output files, and the latter from trees generated from reconstructed elements. See `../scripts/trees` folder for details.

Copy number is calculated from the reconstructed elements, and element length is calculated directly from repbase consensus sequences. Gene distance is calculated (in `distribution.ipynb`) from distances between elements and genes.

In [218]:
# Classes to ignore
ignoreclasses = ['Simple_repeat', 'Satellite', 'Low_complexity', 'rRNA', 'tRNA', 'scRNA', 'snRNA', 'ARTEFACT']

def clean_names(tename):
    tename = tename.strip('-int')  # Remove pointless int tag for some LTRs
    tename = tename.replace('_', '-')  # use only hyphens to match required scRNA format 


### Combining LTRs and internal regions
First we have some tricky steps to handle LTR elements. Since RepeatMasker separates internal and LTR regions for LTR-class TEs, we need to combine these to properly assess mean divergence and copynumber of individual LTR elements. Similarly, since full-length LTR elements comprise a pair of LTRs flanking the internal region, the length calculations need to reflect this.

In [219]:
def get_ltrdict():
    ltrdict = {}
    with open('../data/repeatmasker-out/defragmented/ltrdict.txt') as infile:
        for line in infile:
            line = line.split()
            if len(line) == 2:  # i.e. both internal and LTR regions are present.
                element = commonprefix(line) 
                for suffix in ['-', '_', '_DR', '_DRe']:
                    element = element.strip(suffix)
                # Some LTRs have multiple names; this loop handles these cases
                for rid in line:
                    if ':' in rid:
                        rid = rid.split(':')
                        for i in rid:
                            ltrdict[i] = element
                    else:
                        ltrdict[rid] = element
            elif len(line) == 1:  # i.e. only solo LTRs or Int fragments are present.
                ltrdict[line[0]] = line[0]
    return ltrdict

def reverse_ltrdict():
    ltrdict = get_ltrdict()
    revdict = defaultdict(list)
    for key, val in ltrdict.items():
        revdict[val].append(key)
    return revdict

def handle_multi_ids(teid):
    if ':' in teid:
        teid = teid.split(':')
        for i in teid:
            if i in seqlength_dict.keys():
                return i            
    else:
        return teid
    
def get_ltrlengths():    
    ltr_lengths = {}
    with open('../data/repeatmasker-out/defragmented/ltrdict.txt') as infile:
        for line in infile:
            line = line.split()
            if len(line) == 2:
                internal, ltr = handle_multi_ids(line[0]), handle_multi_ids(line[1])
                ltr_lengths[internal] = seqlength_dict.get(internal, np.nan) + 2*seqlength_dict.get(ltr, np.nan)
                ltr_lengths[ltr] = 0
    return ltr_lengths
    
def map_ltr_lengths(x):
    ltr_lengths = get_ltrlengths()
    if x in ltr_lengths.keys():
        return ltr_lengths[x]
    else:
        return seqlength_dict.get(x, np.nan)
    
ltrlist = []
with open('../data/repeatmasker-out/defragmented/ltrdict.txt') as infile:
    for line in infile:
        line = line.split()
        if len(line) == 2:
            ltrlist.append(line[1])


### Parsed RepeatMasker files from Aurelie Kapusta's `ParseRM.pl` scripts
This gives us information such as genome coverage and mean percent-divergence from consensus sequence.

In [220]:
# Parsed RepeatMasker files from Aurelie Kapusta's `ParseRM.pl` scripts
parserm_df = pd.read_csv('../data/repeatmasker-out/parserm/danRer11.nonalt.fa.align.parseRM.all-repeats.tab', 
                    header=None,
                    skiprows=1,
                    names=['tename', 'teclass', 'tefam', '1', '2', 'frags', '4', 
                           'meandiv', '5', 'meandel', '6', 'meanins', '7', '8', 'cov'],
                    sep='\t')[['tename', 'tefam', 'teclass', 'meandiv', 'meandel', 'meanins', 'cov', 'frags']]

parserm_df = parserm_df.loc[~parserm_df['teclass'].isin(ignoreclasses)]

ltrdict = get_ltrdict()

print(parserm_df.head())
print(parserm_df.shape)
print(len(set(parserm_df['tename'].apply(lambda x: ltrdict.get(x, x)))))

           tename     tefam teclass    meandiv   meandel   meanins       cov  \
0  Gypsy69-LTR_DR     Gypsy     LTR  11.775656  3.173951  2.493011  0.000017   
1  Gypsy88-LTR_DR     Gypsy     LTR  15.967192  1.834374  1.008982  0.000014   
2    Gypsy12-I_DR     Gypsy     LTR   5.864317  1.178092  0.543313  0.000020   
3         KibiDr2    L1-Tx1    LINE   7.772816  3.898475  1.978873  0.000032   
4     DNA-5-8B_DR  hAT-hAT5     DNA  13.801350  8.724557  5.089908  0.000319   

   frags  
0    113  
1    144  
2     52  
3     93  
4   2372  
(2286, 8)
1931


### Calculation of copy number
In order to get a more accurate assessment of copynumber that RepeatMaskers fragment count, we used `one_code_to_find_them_all.pl` to recover fragmented and nested repeats.

In [221]:
# Defragmented elements generated by `one_code_to_find_them_all.pl`

copynum_df = pd.read_csv('../data/repeatmasker-out/defragmented/danRer11.nonalt.fa.out.elem_sorted.csv', 
                        sep='\t')
copynum_df = copynum_df.loc[copynum_df['Score'].str.startswith('###')] \
    .groupby('Element') \
    .agg({'Element': ['count']}) \
    .reset_index()
copynum_df.columns = ['tename', 'copynum']

copynum_df['tename'] = copynum_df['tename'].apply(lambda x: x.strip('-int'))

ltrdict = get_ltrdict()

mergedcopy_df = copynum_df.copy()
mergedcopy_df['tename'] = mergedcopy_df['tename'].apply(lambda x: ltrdict.get(x, x))
mergedcopy_df = mergedcopy_df.groupby('tename').sum().reset_index()
mergedcopy_df.columns = ['tename', 'copynum']

print(mergedcopy_df.head())
print(mergedcopy_df.shape)

/Users/jonwells/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,5,6,7,12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


          tename  copynum
0       ACROBAT1     3277
1       ACROBAT2      201
2          ANGEL    60993
3    Academ-1_DR       36
4  Academ-N1_DRe     2324
(1925, 2)


### Calculation of median distances of elements from nearby genes.
This includes the median distance of insertions from a gene on either strand, and also separately for genes on the same or opposite strands. These distances are calculated using bed files generated from the defragmented insertions found by `one code to find them all`. For further details of the calculation, see `featurecoverage.sh` and `scripts/shuffle_tebed.{sh,py}`.

In [222]:
# Calculate median distance of elements from nearby genes.

def parse_bed(filename):
    """This function parses bed files returned from `bedtools closest`"""
    chroms = [f'chr{i}' for i in range(1,26)]
    
    bed_df = pd.read_csv(filename, 
                         sep='\t',
                         header=None,
                         names=['chrom', 'testart', 'teend', 'tename', 'tefam', 'strand',
                                'chrom2', 'genestart', 'geneend', 'genename', 'dist'],
                         index_col=False)
    bed_df = bed_df[['chrom', 'tename', 'tefam', 'dist']]
    bed_df = bed_df.loc[bed_df['chrom'].isin(chroms)]
    bed_df['teclass'] = bed_df['tefam'].apply(lambda x: x.split('/')[0])
    bed_df['tefam'] = bed_df['tefam'].apply(lambda x: x.split('/')[-1])
    
    # Calculate median distance for each tename
    bed_df = bed_df \
        .groupby(['tename']) \
        .median() \
        .reset_index()
    
    return bed_df

meddist_df = parse_bed('../data/dist/GRCz11_defrag_te_closest_gene.bed')
ss_df = parse_bed('../data/dist/GRCz11_defrag_te_closest_gene_ss.bed').rename({'dist': 'ssdist'}, axis=1)
ds_df = parse_bed('../data/dist/GRCz11_defrag_te_closest_gene_ds.bed').rename({'dist': 'dsdist'}, axis=1)
meddist_df = meddist_df.merge(ss_df, on='tename').merge(ds_df, on='tename')
print(meddist_df.head())
print(meddist_df.shape)

          tename  dist   ssdist   dsdist
0       ACROBAT1   0.0  19259.0  20637.0
1       ACROBAT2   0.0   5653.5  12110.0
2          ANGEL   0.0  12528.5  11574.0
3    Academ-1_DR   0.0  29085.0  24506.0
4  Academ-N1_DRe   0.0   4564.0   6563.0
(1807, 4)


### TE length
Very simple calculation of consensus sequence lengths from RepeatMasker/Repbase libraries. For LTRs, this is calculated as 2*L + I, where L and I are the length of the LTR consensus and Internal consensus, respectively.

In [223]:
# TE lengths from RepeatMasker/Repbase libraries
seqlength_dict = SeqIO.to_dict(SeqIO.parse('../data/misc/zebrep.fa', 'fasta'))
seqlength_dict = {key: len(val.seq) for key, val in seqlength_dict.items()}
length_df = pd.DataFrame \
    .from_dict(seqlength_dict, 
               orient='index', 
               columns=['length']) \
    .reset_index() \
    .rename(columns={'index': 'tename'})

length_df['length'] = length_df['tename'].apply(map_ltr_lengths)
ltrdict = get_ltrdict()
length_df['tename'] = length_df['tename'].apply(lambda x: ltrdict.get(x, x)) 
length_df = length_df.loc[length_df['length'] > 0]

print(length_df.head())
print(length_df.shape)

        tename  length
0   hAT-N38_DR  1029.0
1   TE-X-17_DR   452.0
2    Tx1-49_DR  1583.0
3  DNA-8-30_DR  1022.0
4   hAT-N63_DR  1710.0
(1975, 2)


### TE ages
Since divergence from consensus is innacurate for TEs with significant subfamily structure, we generated trees of insertions for each family and recalculated age as the median terminal branch length of all leaves in the tree. For details see `scripts/trees/` directory.

In [224]:
# Branch length data from pre-computed trees
tree_df = pd.read_csv('../data/trees/trees_summary.txt', sep='\t')
print(tree_df.sort_values('medlen').head())
print(tree_df.shape)

           tename   tefam teclass   meanlen    medlen        25        75  \
1618      L1-2_DR  L1-Tx1    LINE  0.006016  0.000000  0.000000  0.000660   
511   ERV5_DR_LTR    ERV1     LTR  0.002734  0.000000  0.000000  0.000000   
492      Gypsy106   Gypsy     LTR  0.036935  0.000550  0.000550  0.018420   
323        BEL-45     Pao     LTR  0.021318  0.000620  0.000000  0.006205   
1214        BEL19     Pao     LTR  0.016664  0.002315  0.000945  0.012925   

           var  numbranches  
1618  0.001919          910  
511   0.000103           18  
492   0.011029           29  
323   0.007365          102  
1214  0.000937           34  
(1880, 9)


## Cell-type specificity index (tau)
This one is a little more complicated. In order to quantify the degree of cell-type specificity for different TE families, we repurposed a previously described tissue specificity index, $\tau$ (Kryuchkova-Mostacci & Robinson-Rechavi, 2017; Yanai et al., 2005). Rather than using the average expression in a given tissue, we calculated $\tau$ using the average expression in each cluster (CPM/cells per cluster). We applied an additional filter requiring that TEs must be expressed in at least 5% of cells in at least one cluster in order to be labelled as "robustly expressed". This reduces the number of TE families with very high tau values arising as an artifact of low-level stochastic expression in small numbers of cells.

Note that this is the only dataset which also contains gene information. Hence, we have split this into two separated dataframes for TEs and genes respectively.

In [225]:
def calc_tau(x):
    """Calculate tau - a measure of tissue/cell specificity.
    
    Args:
        x - array-like data
    """

    n = len(x)
    max_x = max(x)
    tau = sum([1 - xi/max_x for xi in x])/(n - 1)
    return tau

# Calculation of tau separately for each developmental stage.
dirname = '../data/expression/clustered-ae/'
tau_dfs = []
# This loop extracts measures of tau from each developmental stage and adds to list of dataframes
for filename in os.listdir(dirname):
    if not filename.endswith('_AE.txt'):
        continue
    df = pd.read_csv(f'{dirname}/{filename}', sep='\t', index_col=0)
    stage = filename.split('_')[0]
    df.columns = [f'{stage}_{i}' for i in df.columns]
    df[f'{stage}_tau'] = df.apply(calc_tau, axis=1)  # This here does the actual tau calculation
    tau_dfs.append(df[[f'{stage}_tau']])
    
# Then merge the dataframes and calculate the mean value of tau for every TE family
tau_df = reduce(lambda x, y: x.merge(y, left_index=True, right_index=True, how='outer'), tau_dfs)
tau_df['meantau'] = tau_df.mean(axis=1, skipna=True)
tau_df.reset_index(inplace=True)
tau_df.rename(columns={'index': 'tename'}, inplace=True)
tau_df['tename'] = tau_df['tename'].apply(lambda x: x.strip('-int'))

# Add infor for filtering of robustly expressed genes and TEs
robustgeneexpr = [i.strip().strip('"') for i in open('../data/expression/TE_gene_list_10%_0220.txt').readlines()]
robustgeneexpr = [i for i in robustgeneexpr if i.startswith('ENSDARG')]
robustteexpr5 = [i.strip().strip('"') for i in open('../data/expression/TE.list.morethan5.txt').readlines()] + robustgeneexpr

# Separate TE and genes into separate dataframes, and add some gene types
ribosomal = list(pd.read_csv('../data/misc/Danio_rerio_ribosomal.txt', sep='\t')['Gene stable ID'])
homeobox = pd.read_csv('../data/misc/Danio_rerio_TF.txt', sep='\t')
homeobox = list(homeobox.loc[homeobox['Family'] == 'Homeobox', 'Ensembl'])

gene_tau_df = tau_df.loc[tau_df['tename'].str.startswith('ENSDARG')].copy()
gene_tau_df['tefam'] = 'Gene'
gene_tau_df.loc[gene_tau_df['tename'].isin(ribosomal), 'tefam'] = 'Ribosomal\ngenes'
gene_tau_df.loc[gene_tau_df['tename'].isin(homeobox), 'tefam'] = 'Homeobox\ntranscription factors'
gene_tau_df['teclass'] = gene_tau_df['tefam']
gene_tau_df['robust'] = False
gene_tau_df.loc[gene_tau_df['tename'].isin(robustgeneexpr), 'robust'] = True
gene_tau_df['isltr'] = False

gene_tau_df.to_csv('../data/expression/gene_meantau.txt', sep='\t', index=False)

tau_df = tau_df.loc[~tau_df['tename'].str.startswith('ENSDARG')]

print(tau_df.head())
print(tau_df.shape)

    tename  ZF3S_tau  ZFDOME_tau   ZFB_tau  ZF75_tau  ZF60_tau  ZFHIGH_tau  \
0  (AAATG)  0.864334    0.731162  0.946241  0.862353  0.796940         NaN   
1  (AACTG)  0.937640    0.772971  0.921956  0.800273  0.849347    0.873723   
2  (AAGTG)       NaN         NaN  0.991390       NaN  0.925700         NaN   
3  (AATAG)  0.870893    0.777185  0.852168  0.616807  0.845445    0.596545   
4  (AATTG)  0.946309    0.340420  0.903243  0.863054  0.821763         NaN   

    ZFS_tau  ZF90_tau  ZF30_tau  ZF50_tau  ZFOBLONG_tau  ZF6S_tau   meantau  
0       NaN  0.796188       NaN  0.814403           NaN  0.896568  0.838524  
1  0.752088  0.932411  0.799158  0.904978           NaN  0.874761  0.856300  
2       NaN       NaN       NaN       NaN           NaN       NaN  0.958545  
3  0.711776  0.671990  0.729801  0.590693      0.610043  0.766288  0.719970  
4  0.724970  0.941271  0.899421  0.877614           NaN  0.968718  0.828678  
(2580, 14)


### Sense/antisense ratio
This is the ratio of the total number of sense reads to antisense reads across all cells and stages. It provides an indiciation of the degree to which TE expression is driven by either read-through transcription or internal promoters. A very high sense/antisense ratio is unlikely to be achieved by read-through transcription from neighboring genes as it would require either extensive targeting, strong selection or both, for te insertion downstream of genes. In contrast, a ratio close to 1:1 is most easily explained by random insertion orientation relative to genes, and subsequent read-through transcription in both sense and antisense directions.

In [226]:
antisense_df = pd.read_csv('../data/expression/sense_anti.txt', sep='\t')
print(antisense_df.head())
print(antisense_df.shape)

          tename     tefam teclass    meandiv   meandel   meanins       cov  \
0    Academ-1_DR  Academ-1     DNA   3.701289  1.353107  0.795203  0.000010   
1  Academ-N1_DRe  Academ-1     DNA  16.467405  4.131268  2.755778  0.000229   
2       ACROBAT1       PIF     DNA   6.093923  7.604549  1.702335  0.000542   
3       ACROBAT2       PIF     DNA   9.027897  8.973347  2.816820  0.000059   
4          ANGEL   Kolobok     DNA  16.961762  9.139754  2.392364  0.006887   

   frags  sense/anti  
0     39    5.018405  
1   2236    1.259450  
2   2947    0.962443  
3    282    1.293771  
4  65085    0.883911  
(2286, 9)


### Combining datasets
Finally, we merge together all of the previous dataframes, enabling downstream analyses in separate files. Outer joins are used for all, enabling subsets of this dataset to be used for all downstream analyses.

In [227]:
# Create second column for merged name, then map to either unique or merged name
ltrdict = get_ltrdict()

classes = {}
with open('../data/repeatmasker-out/danRer11_classes_wredundant.txt') as infile:
    for line in infile:
        line = line.strip().split('\t')
        classes[line[0]] = line[1:]

def fix_names(df, tename_col):
    df['mergedname'] = df[tename_col].apply(lambda x: ltrdict.get(x, x))
    df['flatmergedname'] = df['mergedname'].str.replace('_', '-')
    df['flatname'] = df[tename_col].str.replace('_', '-')
    return df

summary_df = fix_names(parserm_df[['tename', 'tefam', 'teclass', 'meandiv', 'cov', 'frags']].copy(), 'tename')

summary_df = summary_df.merge(mergedcopy_df, left_on='mergedname', right_on='tename', how='outer')
summary_df.loc[summary_df.tename_x.isna(), 'tename_x'] = summary_df.loc[summary_df.tename_x.isna(), 'tename_y']
summary_df = fix_names(summary_df, 'tename_x').drop('tename_y', axis=1).rename({'tename_x': 'tename'}, axis=1)

summary_df = summary_df.merge(meddist_df, left_on='mergedname', right_on='tename', how='outer')
summary_df.loc[summary_df.tename_x.isna(), 'tename_x'] = summary_df.loc[summary_df.tename_x.isna(), 'tename_y']
summary_df = fix_names(summary_df, 'tename_x').drop('tename_y', axis=1).rename({'tename_x': 'tename'}, axis=1)

summary_df = summary_df.merge(length_df, left_on='mergedname', right_on='tename', how='outer')
summary_df.loc[summary_df.tename_x.isna(), 'tename_x'] = summary_df.loc[summary_df.tename_x.isna(), 'tename_y']
summary_df = fix_names(summary_df, 'tename_x').drop('tename_y', axis=1).rename({'tename_x': 'tename'}, axis=1)

summary_df = summary_df.merge(tree_df[['tename', 'medlen']], left_on='mergedname', right_on='tename', how='outer')
summary_df.loc[summary_df.tename_x.isna(), 'tename_x'] = summary_df.loc[summary_df.tename_x.isna(), 'tename_y']
summary_df = fix_names(summary_df, 'tename_x').drop('tename_y', axis=1).rename({'tename_x': 'tename'}, axis=1)

summary_df.loc[summary_df.teclass.isna(), 'tefam'] = summary_df.loc[summary_df.teclass.isna()].apply(lambda x: classes['tename'][0])
summary_df = summary_df.loc[~summary_df.tefam.isna()]

summary_df.loc[summary_df['tefam'] == 'RC', 'teclass'] = 'RC'

# # Mark LTR regions
summary_df['isltr'] = False
summary_df.loc[summary_df.tename.isin(ltrlist), 'isltr'] = True

# Just in case
summary_df = summary_df.drop_duplicates().sort_values('medlen')

print(summary_df)
print(summary_df.loc[summary_df.tename.isna()])

summary_df.to_csv('../data/danrer11_te_summary.txt', sep='\t', index=False, na_rep='nan')

               tename    tefam  teclass    meandiv       cov   frags  \
175           L1-2_DR   L1-Tx1     LINE   7.173071  0.000335  6103.0   
833       ERV5_DR_LTR     ERV1      LTR   4.491648  0.000006    20.0   
872     Gypsy106-I_DR    Gypsy      LTR   1.335491  0.000034    44.0   
871   Gypsy106-LTR_DR    Gypsy      LTR   3.645278  0.000023    45.0   
425    BEL-45_DRe-LTR      Pao      LTR  12.166720  0.000022   189.0   
...               ...      ...      ...        ...       ...     ...   
2203       TE-X-25_DR  Unknown  Unknown  10.470897  0.000018   111.0   
2225       TE-X-26_DR  Unknown  Unknown  16.419407  0.000070   637.0   
2232      hAT-N196_DR     hAT?      DNA  17.778350  0.000003    30.0   
2262   Gypsy74-LTR_DR    Gypsy      LTR   6.374631  0.000007    38.0   
2263     Gypsy74-I_DR    Gypsy      LTR   3.779143  0.000014    34.0   

       mergedname flatmergedname         flatname  copynum     dist   ssdist  \
175       L1-2_DR        L1-2-DR          L1-2-DR   317